In [4]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [5]:
train_path = 'resources/train'
valid_path = 'resources/valid'
test_path = 'resources/test'

In [3]:
train_batches = ImageDataGenerator(rotation_range = 90, horizontal_flip=True, vertical_flip=True).flow_from_directory(train_path, target_size=(224,224), classes=['potato', 'catfood', 'table'], batch_size = 10)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size=(224,224), classes=['potato', 'catfood', 'table'], batch_size = 5)
test_batches  = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), classes=['potato', 'catfood', 'table'], batch_size = 5)

Found 299 images belonging to 3 classes.
Found 82 images belonging to 3 classes.
Found 35 images belonging to 3 classes.


In [6]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

valid_datagen = ImageDataGenerator(
        rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
        train_path,  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=batch_size,
        classes=['potato', 'catfood', 'table'])  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = valid_datagen.flow_from_directory(
        valid_path,
        target_size=(224, 224),
        batch_size=batch_size,
        classes=['potato', 'catfood', 'table'])

Found 299 images belonging to 3 classes.
Found 82 images belonging to 3 classes.


In [7]:
from keras.applications import mobilenet
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import models
from keras import layers

In [8]:
base_model = mobilenet.MobileNet(weights='imagenet', include_top = False, input_shape=(224, 224, 3))

In [9]:
#from keras.utils import plot_model
#plot_model(base_model, to_file='model_NoTop.png')

In [10]:
for layer in base_model.layers:
    layer.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
preds=Dense(3,activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=base_model.input,outputs=preds)

model.summary()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [7]:
# add a global spatial average pooling layer
model = models.Sequential()
for layer in base_model.layers:
    layer.trainable = False
model.add(base_model)

# Add new layers
model.add(layers.AveragePooling2D())

model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(3, activation='softmax'))


    
# Show a summary of the model. Check the number of trainable parameters
model.summary()

# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
# x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
# predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
# model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
# for layer in base_model.layers:
#    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, 7, 7, 1024)        3228864   
_________________________________________________________________
average_pooling2d_1 (Average (None, 3, 3, 1024)        0         
_________________________________________________________________
dense_1 (Dense)              (None, 3, 3, 1024)        1049600   
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 3, 1024)        0         
_________________________________________________________________
dense_2 (Dense)              (None, 3, 3, 1024)        1049600   
_________________________________________________________________
dropout_2 (Dropout)          (None, 3, 3, 1024)        0         
_________________________________________________________________
dense_3 (Dense)              (None, 3, 3, 512)         524800    
__________

In [11]:
model.fit_generator(train_generator, steps_per_epoch=18, epochs=50, validation_data=validation_generator, validation_steps=2)

Epoch 1/50
18/18 [==============================] - 3s 169ms/step - loss: 1.5084 - acc: 0.8436 - val_loss: 0.7239 - val_acc: 0.7500
Epoch 2/50
18/18 [==============================] - 3s 139ms/step - loss: 0.1665 - acc: 0.9652 - val_loss: 0.4678 - val_acc: 0.8438
Epoch 3/50
18/18 [==============================] - 3s 147ms/step - loss: 0.4435 - acc: 0.9359 - val_loss: 0.2738 - val_acc: 0.8333
Epoch 4/50
18/18 [==============================] - 2s 136ms/step - loss: 0.1551 - acc: 0.9479 - val_loss: 0.4520 - val_acc: 0.8438
Epoch 5/50
18/18 [==============================] - 3s 139ms/step - loss: 0.0899 - acc: 0.9757 - val_loss: 0.3883 - val_acc: 0.8125
Epoch 6/50
18/18 [==============================] - 2s 137ms/step - loss: 0.7225 - acc: 0.9305 - val_loss: 0.3436 - val_acc: 0.8333
Epoch 7/50
18/18 [==============================] - 3s 141ms/step - loss: 0.0678 - acc: 0.9896 - val_loss: 0.8551 - val_acc: 0.8125
Epoch 8/50
18/18 [==============================] - 3s 142ms/step - loss: 0.

In [12]:
test_datagen = ImageDataGenerator(
        rescale=1./255
)

# this is a similar generator, for validation data
test_generator = valid_datagen.flow_from_directory(
        test_path,
        target_size=(224, 224),
        batch_size=batch_size,
        classes=['potato', 'catfood', 'table'])

Found 35 images belonging to 3 classes.


In [14]:
imgs, labels = next(test_generator)

ynew = model.predict(imgs)

In [15]:
np.round(ynew, decimals=2)

array([[1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [0.  , 0.07, 0.93],
       [0.  , 0.23, 0.77],
       [0.  , 1.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [0.  , 0.01, 0.99],
       [0.  , 0.09, 0.91],
       [0.  , 1.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  ],
       [0.  , 0.  , 1.  ],
       [0.  , 0.33, 0.67]], dtype=float32)

In [16]:
labels

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [17]:
model.evaluate(imgs,labels, batch_size=batch_size)

16/16 [==============================] - 0s 4ms/step


[0.05216670036315918, 1.0]

In [25]:
from keras.preprocessing import image

gul_path = 'tfpoet/tf_files/test/gulerod.jpg'
img = image.load_img(gul_path, target_size=(224, 224))
img = image.img_to_array(img)
img = img
img = np.expand_dims(img, axis=0)
out = model.predict(img)
print(np.round(out, decimals = 2))

[[0. 0. 1.]]
